<a href="https://colab.research.google.com/github/ebtesam25/lendsba/blob/main/FintechLoan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install streamlit


     |████████████████████████████████| 7.0MB 5.6MB/s 
     |████████████████████████████████| 163kB 48.3MB/s 
     |████████████████████████████████| 4.6MB 45.9MB/s 
     |████████████████████████████████| 112kB 60.2MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 122kB 55.6MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=f3065e37e351051cdc4a54bcd0d14e65d70455e2d95560dd14f1a9265b95a07b
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.3 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


In [ ]:
pip install lime

     |████████████████████████████████| 276kB 5.5MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp37-none-any.whl size=283846 sha256=2c4796681a31f0b039fab90da461d81f0819f91c2a2b2c81fd71fe902ed3e0f7
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime


In [ ]:
pip install dill

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-04-04 19:04:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.224.116.172, 34.235.3.193, 52.200.34.95, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.224.116.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.06M  71.9MB/s    in 0.2s    

2021-04-04 19:04:28 (71.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14746350/14746350]



In [ ]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score
import pickle


#simplefilter(action='ignore', category=FutureWarning)
from imblearn.under_sampling import RandomUnderSampler

df = pd.read_csv('Trail.csv')

v1=df.dropna()
y = v1['LoanStatus']
X = v1.drop(['LoanStatus'], axis = 1)

x_train, x_test, y_train, y_test = train_test_split( X, y, test_size=.20, random_state=0)
undersample = RandomUnderSampler(sampling_strategy='majority')
X_over, y_over = undersample.fit_resample(x_train, y_train)
import pickle
clf3 = RandomForestClassifier(n_estimators= 100)
clf3.fit(X_over, y_over)
filename = 'finalized_model.sav'
pickle.dump(clf3, open(filename, 'wb'))


import lime
import lime.lime_tabular
X_featurenames = x_train.columns
explainer = lime.lime_tabular.LimeTabularExplainer(np.array(X_over),
                    feature_names=X_featurenames, class_names=['0','1'], discretize_continuous=True)
import dill
#pickle.dump(explainer, open(filename1, 'wb'))
with open('Explanation', 'wb') as f:
    dill.dump(explainer, f)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
%%writefile app.py

import pandas as pd
import streamlit as st
import pandas as pd
import dill
import pickle
import numpy as np
import lime
import lime.lime_tabular

st.write("""
# Loan Approval Prediction App
This app predicts whether loan would be approved or not and features affecting the decision
""")


def user_input_features():
    a=st.text_input('Enter age')
    b=st.text_input('Enter Income')
    c=st.text_input('Enter Credit Score')
    d=st.text_input('Enter House hold size')
    e=st.text_input('Enter Median Home Value')
    f=st.text_input('Enter Median Household Income')
    g=st.text_input('Enter Debt')
    h=st.text_input('Enter Loan Term')
    i=st.text_input('Enter Interest Rate')
    j=st.text_input('Enter Credit Incidents')
    k=st.text_input('Enter HomeValue')
    l=st.text_input('Enter Loan Amount')
    m=st.selectbox('Select Loan type', ['Government_insured','Fixed_rate','Adjustable_rate','Jumbo'])
    st.text("")
    st.text("")
    st.text("")
    n=1   
    if(m=='Government_insured'):
        n=1
    elif(m=='Fixed_rate'):
        n=2
    elif(m=='Adjustable_rate'):
        n=3
    elif(m=='Jumbo'):
        n=4   
    data = {'Age': int(a),
                'Income': int(b),
                'CreditScore': int(c),
                'HouseholdSize': int(d),
                'MedianHomeValue': int(e),
                'MedianHouseholdIncome': int(f),
                'Debt': float(g),
                'LoanTerm': int(h),
                'InterestRate': float(i),
                'CreditIncidents': int(j),
                'HomeValue': int(k),
                'LoanAmount': int(l),
                'ProductType': int(n)     
                }
    features = pd.DataFrame(data, index=[0])
    return features


def please(testing):
    testing=testing
    result = loaded_model.predict(testing)
    expg = explaining.explain_instance(testing.iloc[0], loaded_model.predict_proba)
    expg.save_to_file('will.html')
    return result

filename = 'finalized_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
filename1='Explanation'
explaining = dill.load(open(filename1, 'rb'))




testing = user_input_features()
ma=st.button('Predict!!')
st.text("")
st.text("")
if ma:
    m1=please(testing)
    if(m1==1):
        st.write('Hurrah! There are strong chances that your loan would be accepted.')
    elif(m1==0):
        st.write('Sorry! There are not any bright chance of getting your loan accepted')
    st.text("")
    st.text("")
    st.write('Below is the description about features affecting the decision')
    st.write('1 depicts loan approaval and 0 depicts disapproval')
    
    st.text("")
    st.text("")
    st.text("")
    import streamlit.components.v1 as components
    HtmlFile = open("will.html", 'r', encoding='utf-8')
    source_code = HtmlFile.read() 
    print(source_code)
    components.html(source_code, height=1000)

Overwriting app.py


In [ ]:
!ls

app.py	     finalized_model.sav  ngrok-stable-linux-amd64.zip	Trail.csv
Explanation  ngrok		  sample_data


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://9cb6115adf08.ngrok.io


In [ ]:
!streamlit run app.py

Streaming output truncated to the last 5000 lines.
	  ];
	});


/***/ }),
/* 217 */
/***/ (function(module, exports, __webpack_require__) {

	// 7.3.20 SpeciesConstructor(O, defaultConstructor)
	var anObject = __webpack_require__(19);
	var aFunction = __webpack_require__(31);
	var SPECIES = __webpack_require__(34)('species');
	module.exports = function (O, D) {
	  var C = anObject(O).constructor;
	  var S;
	  return C === undefined || (S = anObject(C)[SPECIES]) == undefined ? D : aFunction(S);
	};


/***/ }),
/* 218 */
/***/ (function(module, exports, __webpack_require__) {

	'use strict';
	var LIBRARY = __webpack_require__(29);
	var global = __webpack_require__(11);
	var ctx = __webpack_require__(30);
	var classof = __webpack_require__(82);
	var $export = __webpack_require__(15);
	var isObject = __webpack_require__(20);
	var aFunction = __webpack_require__(31);
	var anInstance = __webpack_require__(219);
	var forOf = __webpack_require__(220);
	var speciesConstructor = __webpack_requir